# Чтение данных сайта ЦБР

In [1]:
from os import path, remove
import urllib.parse as urlparse
from urllib.parse import parse_qs, urljoin
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
import sqlalchemy as sa
import locale
from cmasf import pandas_sql as pds
import tabula

from IPython.display import Markdown as md

In [2]:
# пути к файлам
strBasePath=path.join('..', 'DB', 'bankrp.sqlite3') # путь к рабочей базе данных SQLite - может быть изменено в вызывающем блокноте
# ------------

# константы базы данных SQLite3
strLOAN_table='loan' # название таблицы LOAN в базе данных SQLite
strLOAN_data_pass='loan_columns' # таблица с названиями колонок для таблицы loan
strLendTable='lending'
strLendPRC='lending_persc'
strLendReserv='lending_reserv'

strInfoTable='update_info'

# словарь для конверсии русских дат в нормальные
dct_month={'январь':'Jan', 'февраль':'Feb', 'март':'Mar', 'апрель':'Apr', 'май':'May', 'июнь':'Jun', 
           'июль':'Jul', 'август':'Aug', 'сентябрь':'Sep', 'октябрь':'Oct', 'ноябрь':'Nov', 'декабрь':'Dec'}


conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strBasePath)) # connection к рабочей базе данных

def iterate_group(iterator, count):
    itr = iter(iterator)
    for i in range(0, len(iterator), count):
        yield iterator[i:i + count]

## Обработка базы данных кредитной ставки с сайта ЦБР

**Источник - http://cbr.ru/statistics/bank_sector/int_rat/**  
На странице источника ищем ссылки с ключевым текстом "Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям" 

In [3]:
strCBR_LOAN_SRC=r'http://cbr.ru/statistics/bank_sector/int_rat/'

strKeyText='Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям'.replace(' ', '\s+')

re_t=re.compile(strKeyText)

resp=requests.get(strCBR_LOAN_SRC)

loan_soup=BeautifulSoup(resp.text)

div_docs=loan_soup.findAll('div', class_='document-regular')

print('Найдено инфо-тегов', len(div_docs))

Найдено инфо-тегов 31


Читаем файл по сылке, искомая ставка в столбце `F` листа `ставки_руб`.  
Преобразуем даты в формат datetime

In [4]:
def find_link(div_tags):
    for d in div_tags:
        if d.find('span', class_='document-regular_name_visible', text=re_t) and d.find('div', class_='document-regular_comment', text=re.compile('в целом по Российской Федерации'.replace(' ', '\s'))):
            pub_date=dt.datetime.strptime(d.find('div', class_='document-regular_date').text.strip(), '%d.%m.%Y')
            a=d.find('a')
            
            print('Найден файл с нужными данныи, дата публикации {date}, ссылка - {link}'.format(date=pub_date.date(), link=a['href']))
            return pub_date, a['href']

loan_pub_date, loan_href=find_link(div_docs)

Найден файл с нужными данныи, дата публикации 2020-10-12, ссылка - /vfs/statistics/pdko/int_rat/loans_nonfin.xlsx


In [5]:
def convert_date(x):
    dct_month_transl={'Январь':1, 'Февраль':2, 'Март':3, 'Апрель':4, 'Май':5, 'Июнь':6, 
                  'Июль':7, 'Август':8, 'Сентябрь':9, 'Октябрь':10, 'Ноябрь':11, 'Декабрь':12}
    dtp=x.split(' ')
    return pd.to_datetime(dt.date(int(dtp[1]), dct_month_transl[dtp[0]], 1))

pdf_loan=pd.read_excel(urljoin(strCBR_LOAN_SRC, loan_href), sheet_name='ставки_руб.', 
                       skiprows=4, usecols='A, F').dropna()

pdf_loan.columns=['date', 'loan_nonfin']
pdf_loan['date']=pdf_loan['date'].apply(convert_date)
pdf_loan.set_index('date', inplace=True)
pdf_loan

,loan_nonfin
date,
2014-01-01,9.15
2014-02-01,9.43
2014-03-01,10.29
2014-04-01,10.53
2014-05-01,10.60
...,...
2020-04-01,7.71
2020-05-01,7.28
2020-06-01,6.89


### Запись в базу данных и обновление таблицы апдейтов

In [7]:
_pdf=pds.DataFrameDATA(pdf_loan.copy())
_pdf.to_sql(strLOAN_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

pds.DataFrameDATA([{'index':'date', 'col_name': '''Средневзвешенные процентные ставки по кредитам, 
предоставленным кредитными организациями нефинансовым организациям в рублях, до 1 года, 
включая "до востребования"'''}] ).set_index('index').to_sql(strLOAN_data_pass, con=conWork, if_exists='replace')

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLOAN_table))

In [8]:
pdf_loan.groupby(pd.Grouper(freq='Y')).mean()

,loan_nonfin
date,
2014-12-31,12.040000
2015-12-31,16.930000
2016-12-31,14.150000
2017-12-31,11.917500
2018-12-31,9.988333
2019-12-31,9.826667
2020-12-31,8.625714


## Читаем данные из "Обзор банковского сектора Российской Федерации"

**Источник - http://cbr.ru/statistics/bank_sector/ **

Читаем только Excel-файлы (ист. данные из pdf-файлов уже скачены и лежат в базе).  
Выбираем файл с последней датой, экспресс-обзор игнорируется.  
Читаем таблицы 47 и 51, из 47-й читаем и объемы и проценты, из 51-й читаем только "Фактически сформированный резерв, в % от ссудной задолженности данной категории качества"

Есть возможность прочитать несколько файлов (надо код изменить)

In [4]:
strURL_=r'http://cbr.ru/statistics/bank_sector/review/'


resp=requests.get(strURL_)
ln2_soup=BeautifulSoup(resp.text)

print(resp)

<Response [200]>


Строим список нужный файлов с датой публикацией и текстом ссылки

In [17]:
strCBR=r'http://cbr.ru'

a_refs=ln2_soup.findAll('a', class_='versions_item')

lst_xsl_hrefs=[]

for a in a_refs:
    try:
        iyear=re.search(r'(?m)(?P<year>\d{4})', a['title']).group('year') 
        strSM=r'(?m)(?P<month>({months}))'.format(months=')|('.join(dct_month.keys()))
        str_month=re.search(strSM, a.text).group('month') 
#         print(str_month)
    except:
        print('some error - ', a['title'], a['href'], a.text)
        continue
        
    dtpub=dt.datetime.strptime('1 {month} {year}'.format(month=dct_month[str_month], year= iyear), '%d %b %Y')
    
    if re.search(r'\.xls\w', a['href']) and not re.search('экспресс-выпуск', a.text):
        lst_xsl_hrefs.append({'url':strCBR + a['href'], 'date':dtpub, 'title':a.text})
        
pdf_files=pd.DataFrame(lst_xsl_hrefs).sort_values(by='date', ascending=False)
print('='*70)
pdf_files

some error -  Опубликовано 22 сентября 2020 /Collection/Collection/File/29261/obs_216.xlsx экспресс-выпуск (на 01.09.2020)
some error -  Опубликовано 21 августа 2020 /Collection/Collection/File/29170/obs_215.xlsx экспресс-выпуск (на 01.08.2020)
some error -  Опубликовано 03 декабря 2019 /Collection/Collection/File/25496/Metodol_21.pdf Выпуск 21
some error -  Опубликовано 17 августа 2016 /Collection/Collection/File/8547/Metodol_20.pdf Выпуск 20
some error -  Опубликовано 01 апреля 2014 /Collection/Collection/File/8548/Metodol_19.pdf Выпуск 19
some error -  Опубликовано 18 марта 2013 /Collection/Collection/File/8549/Metodol_18.pdf Выпуск 18
some error -  Опубликовано 06 августа 2012 /Collection/Collection/File/8550/Metodol_17.pdf Выпуск 17


,url,date,title
0,http://cbr.ru/Collection/Collection/File/28028...,2020-08-01,2020 г.: август
1,http://cbr.ru/Collection/Collection/File/27973...,2020-07-01,2020 г.: июль
2,http://cbr.ru/Collection/Collection/File/27902...,2020-06-01,2020 г.: июнь
3,http://cbr.ru/Collection/Collection/File/27838...,2020-05-01,2020 г.: май
4,http://cbr.ru/Collection/Collection/File/27388...,2020-03-01,2020 г.: март
5,http://cbr.ru/Collection/Collection/File/25668...,2020-01-01,2020 г.: январь


#### Читаем данные по выбранной ссылке (по споследней дате, можно изменить)

Читаем три таблицы в три фрейма

In [34]:
iLoc=0

print('Для обновления берем файл (один) с последней датой - ', pdf_files.iloc[iLoc, 1])

href=pdf_files.iloc[iLoc, 0]
print('Читаем по ссылке - ', href)

def read_excel_bank_review(strUrl):
    lst_head=['standart', 'nonsnandart', 'doubtful', 'problem', 'hopeless']
    
    def upwork(pdf, lstHead=lst_head):
        _p=pdf.T
        _p.columns=lstHead
        _p.index.name='date'
        _p=_p.astype(float)
        return _p
        
    def read_t51(strUrl):
        _p=pd.read_excel(strUrl, sheet_name='T51', header=3,
                        usecols='A:K', nrows=4).rename(columns={'Unnamed: 0':'row_head'}).set_index('row_head')
        _p.columns=[re.sub('^0', '', pd.to_datetime(c).strftime('%d.%m.%y'), count=1) for c in _p.columns]
        _p=_p.iloc[:, _p.shape[1]//2:]
        
        _p=upwork(_p, lstHead=lst_head[1:])
        return pds.DataFrameDATA(_p)
    
    def read_t47(strUrl):
        _p=pd.read_excel(strUrl, sheet_name='T47', header=5, 
                        usecols='C:M', nrows=5, 
                        skiprows=[6,] ).rename(columns={'Unnamed: 2':'row_head'}).set_index('row_head')

        lst_new_col=[c for c in _p.columns if not re.search('Unnamed\:', str(c))]
        _p1 = _p[[c for c in _p.columns if c not in lst_new_col]]
        _p  = _p[lst_new_col]
        _p.columns=[re.sub('^0', '', c.strftime('%d.%m.%y'), count=1) for c in _p.columns]
        _p1.columns=_p.columns
        
        _p=upwork(_p)
        _p1=upwork(_p1)
        return pds.DataFrameDATA(_p1), pds.DataFrameDATA(_p)
    
    t47_prc, t47=read_t47(strUrl)
    t51=read_t51(strUrl)
    
    return t47_prc, t47, t51


p47_prct, p47, p51=read_excel_bank_review(href)

print(p47_prct)
print(p47)
print(p51)

Для обновления берем файл (один) с последней датой -  2020-08-01 00:00:00
Читаем по ссылке -  http://cbr.ru/Collection/Collection/File/28028/obs_214.xlsx
          standart  nonsnandart  doubtful   problem  hopeless
date                                                         
1.01.19  43.994174    39.234376  6.647852  2.935216  7.188381
1.01.20  41.445623    42.533939  6.727371  2.478985  6.814082
1.04.20  42.009436    41.885082  6.658417  2.605955  6.841110
1.06.20  41.177163    41.966653  7.196352  2.679141  6.980691
1.07.20  40.588374    42.634103  7.201736  2.631722  6.944064
             standart   nonsnandart     doubtful      problem     hopeless
date                                                                      
1.01.19  27680.139319  24685.382276  4182.677905  1846.771835  4522.766934
1.01.20  27103.765896  27815.480532  4399.429346  1621.156486  4456.134548
1.04.20  29156.661641  29070.353647  4621.276250  1808.663571  4748.074319
1.06.20  28197.429752  28738.058949  

### Запись в базу данных информации по ссудам и резервам, обновление таблицы апдейтов

In [32]:
p47.to_sql(strLendTable, con=conWork, if_exists='upsert')
_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLendTable))

p47_prct.to_sql(strLendPRC, con=conWork, if_exists='upsert')
_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLendPRC))

p51.to_sql(strLendReserv, con=conWork, if_exists='upsert')
_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTable, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLendReserv))


# Завершение работы, сжатие базы данных

In [8]:
import sqlite3
conn = sqlite3.connect(strBasePath, isolation_level=None)
conn.execute("VACUUM") # сжатие базы данных
conn.close()